In [3]:
import tensorflow as tf
import datetime, os
import numpy as np
import pandas as pd


from tensorboard.plugins.hparams import api as hp

from tensorflow import feature_column
from tensorflow.keras import layers

In [4]:
tf.__version__

'2.0.0'

In [5]:
%load_ext tensorboard

In [6]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train=x_train.reshape(60000, 28, 28, 1)
x_train=x_train / 255.0
x_test = x_test.reshape(10000, 28, 28, 1)
x_test=x_test/255.0

In [7]:
HP_NUM_UNITS1 = hp.HParam('num_units 1', hp.Discrete([4,8,16])) 
HP_NUM_UNITS2 = hp.HParam('num_units 2', hp.Discrete([4,8]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.2, 0.5))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam','RMSprop']))
HP_L2 = hp.HParam('l2 regularizer', hp.RealInterval(.001,.01))
METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS1,HP_NUM_UNITS2, HP_DROPOUT,HP_L2 ,HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [8]:
def train_model(hparams):
  
    model =  tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS1], activation='relu'),
    layers.Dropout(hparams[HP_DROPOUT]),
    tf.keras.layers.Dense(10, activation='softmax')])
    
    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    logdir = os.path.join("log1s", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

    model.fit(x=x_train, 
            y=y_train, 
            epochs=10, 
            validation_data=(x_test, y_test))#, 
#            callbacks=[tensorboard_callback])
    _, accuracy = model.evaluate(x_test, y_test)
    return accuracy
#train_model()

In [9]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [10]:
import time
start_time = time.time()
session_num = 0
for num_units1 in HP_NUM_UNITS1.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
       #for optimizer in HP_OPTIMIZER.domain.values:
        hparams = {
            HP_NUM_UNITS1: num_units1,
            HP_DROPOUT: dropout_rate     
        }
        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('logs/hparam_tuning/' + run_name, hparams)
        session_num += 1
print("--- %s seconds ---" % (time.time() - start_time))

--- Starting trial: run-0
{'num_units 1': 4, 'dropout': 0.2}
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 105s 2ms/sample - loss: 1.5817 - accuracy: 0.3834 - val_loss: 1.1041 - val_accuracy: 0.7087
Epoch 2/10
60000/60000 [==============================] - 106s 2ms/sample - loss: 1.1539 - accuracy: 0.5874 - val_loss: 0.6678 - val_accuracy: 0.7982
Epoch 3/10
60000/60000 [==============================] - 111s 2ms/sample - loss: 0.9575 - accuracy: 0.6635 - val_loss: 0.5876 - val_accuracy: 0.8190
Epoch 4/10
60000/60000 [==============================] - 105s 2ms/sample - loss: 0.9038 - accuracy: 0.6868 - val_loss: 0.5643 - val_accuracy: 0.8295
Epoch 5/10
60000/60000 [==============================] - 91s 2ms/sample - loss: 0.8805 - accuracy: 0.6951 - val_loss: 0.5502 - val_accuracy: 0.8306
Epoch 6/10
60000/60000 [==============================] - 92s 2ms/sample - loss: 0.8657 - accuracy: 0.6993 - val_loss: 0.5525 - val_accuracy

--- Starting trial: run-1
{'num_units 1': 4, 'dropout': 0.5}
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 100s 2ms/sample - loss: 1.7857 - accuracy: 0.3009 - val_loss: 1.0517 - val_accuracy: 0.7190
Epoch 2/10
60000/60000 [==============================] - 106s 2ms/sample - loss: 1.5328 - accuracy: 0.3981 - val_loss: 0.9159 - val_accuracy: 0.7341
Epoch 3/10
60000/60000 [==============================] - 119s 2ms/sample - loss: 1.5155 - accuracy: 0.4004 - val_loss: 0.9163 - val_accuracy: 0.7392
Epoch 4/10
60000/60000 [==============================] - 92s 2ms/sample - loss: 1.4881 - accuracy: 0.4079 - val_loss: 0.8492 - val_accuracy: 0.7517
Epoch 5/10
60000/60000 [==============================] - 94s 2ms/sample - loss: 1.4889 - accuracy: 0.4035 - val_loss: 0.8686 - val_accuracy: 0.7496
Epoch 6/10
60000/60000 [==============================] - 92s 2ms/sample - loss: 1.4778 - accuracy: 0.4042 - val_loss: 0.8592 - val_accuracy:

--- Starting trial: run-2
{'num_units 1': 8, 'dropout': 0.2}
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 126s 2ms/sample - loss: 1.1281 - accuracy: 0.5768 - val_loss: 0.6209 - val_accuracy: 0.7813
Epoch 2/10
60000/60000 [==============================] - 100s 2ms/sample - loss: 0.7379 - accuracy: 0.6973 - val_loss: 0.4787 - val_accuracy: 0.8291
Epoch 3/10
60000/60000 [==============================] - 102s 2ms/sample - loss: 0.6689 - accuracy: 0.7308 - val_loss: 0.4565 - val_accuracy: 0.8402
Epoch 4/10
60000/60000 [==============================] - 117s 2ms/sample - loss: 0.6433 - accuracy: 0.7456 - val_loss: 0.4327 - val_accuracy: 0.8458
Epoch 5/10
60000/60000 [==============================] - 106s 2ms/sample - loss: 0.6115 - accuracy: 0.7638 - val_loss: 0.3980 - val_accuracy: 0.8673
Epoch 6/10
60000/60000 [==============================] - 108s 2ms/sample - loss: 0.5738 - accuracy: 0.7854 - val_loss: 0.3872 - val_accura

--- Starting trial: run-3
{'num_units 1': 8, 'dropout': 0.5}
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 103s 2ms/sample - loss: 1.5629 - accuracy: 0.3726 - val_loss: 0.8219 - val_accuracy: 0.7859
Epoch 2/10
60000/60000 [==============================] - 92s 2ms/sample - loss: 1.4262 - accuracy: 0.4144 - val_loss: 0.7300 - val_accuracy: 0.8078
Epoch 3/10
60000/60000 [==============================] - 101s 2ms/sample - loss: 1.3927 - accuracy: 0.4264 - val_loss: 0.6973 - val_accuracy: 0.8207
Epoch 4/10
60000/60000 [==============================] - 126s 2ms/sample - loss: 1.3689 - accuracy: 0.4207 - val_loss: 0.6581 - val_accuracy: 0.8222
Epoch 5/10
60000/60000 [==============================] - 120s 2ms/sample - loss: 1.3559 - accuracy: 0.4108 - val_loss: 0.6717 - val_accuracy: 0.8148
Epoch 6/10
60000/60000 [==============================] - 106s 2ms/sample - loss: 1.3333 - accuracy: 0.4166 - val_loss: 0.6341 - val_accurac

--- Starting trial: run-4
{'num_units 1': 16, 'dropout': 0.2}
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 104s 2ms/sample - loss: 0.8774 - accuracy: 0.6690 - val_loss: 0.5526 - val_accuracy: 0.8192
Epoch 2/10
60000/60000 [==============================] - 107s 2ms/sample - loss: 0.6804 - accuracy: 0.7457 - val_loss: 0.4355 - val_accuracy: 0.8538
Epoch 3/10
60000/60000 [==============================] - 108s 2ms/sample - loss: 0.6099 - accuracy: 0.7642 - val_loss: 0.3956 - val_accuracy: 0.8638
Epoch 4/10
60000/60000 [==============================] - 108s 2ms/sample - loss: 0.5528 - accuracy: 0.7873 - val_loss: 0.3685 - val_accuracy: 0.8777
Epoch 5/10
60000/60000 [==============================] - 107s 2ms/sample - loss: 0.5164 - accuracy: 0.8072 - val_loss: 0.3565 - val_accuracy: 0.8806
Epoch 6/10
60000/60000 [==============================] - 108s 2ms/sample - loss: 0.4887 - accuracy: 0.8178 - val_loss: 0.3549 - val_accur

--- Starting trial: run-5
{'num_units 1': 16, 'dropout': 0.5}
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 96s 2ms/sample - loss: 1.1447 - accuracy: 0.5549 - val_loss: 0.5324 - val_accuracy: 0.8102
Epoch 2/10
60000/60000 [==============================] - 94s 2ms/sample - loss: 0.8884 - accuracy: 0.6559 - val_loss: 0.4918 - val_accuracy: 0.8233
Epoch 3/10
60000/60000 [==============================] - 94s 2ms/sample - loss: 0.8315 - accuracy: 0.6791 - val_loss: 0.4499 - val_accuracy: 0.8464
Epoch 4/10
60000/60000 [==============================] - 92s 2ms/sample - loss: 0.7927 - accuracy: 0.6952 - val_loss: 0.4337 - val_accuracy: 0.8636
Epoch 5/10
60000/60000 [==============================] - 114s 2ms/sample - loss: 0.7727 - accuracy: 0.7034 - val_loss: 0.4163 - val_accuracy: 0.8660
Epoch 6/10
60000/60000 [==============================] - 107s 2ms/sample - loss: 0.7507 - accuracy: 0.7142 - val_loss: 0.3864 - val_accuracy:

--- 6319.41582775116 seconds ---


In [13]:
%tensorboard --logdir logs/hparam_tuning
#%tensorboard --logdir log1s

Reusing TensorBoard on port 6006 (pid 12912), started 18:30:09 ago. (Use '!kill 12912' to kill it.)

In [75]:
%reload_ext tensorboard

In [56]:
import time
start_time = time.time()

print("--- %s seconds ---" % (datetime..time() - datetime.time()))

TypeError: unsupported operand type(s) for -: 'datetime.time' and 'datetime.time'